In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import kagglehub
import os
import shutil


# Define the path where you want to store the dataset
dataset_folder_path = '/content/drive/MyDrive/Datasets'
cifar100_dataset_folder_path = os.path.join(dataset_folder_path, 'cifar100')


In [5]:


# Create the directory if it doesn't exist
os.makedirs(cifar100_dataset_folder_path, exist_ok=True)

# Download the dataset
print("Downloading dataset...")
path = kagglehub.dataset_download("ibraheemmoosa/cifar100-256x256")
print("Path to dataset files:", path)

# Move the downloaded files to the desired directory
for file_name in os.listdir(path):
    source_file_path = os.path.join(path, file_name)
    destination_file_path = os.path.join(cifar100_dataset_folder_path, file_name)

    # Move the file
    shutil.move(source_file_path, destination_file_path)

print("Dataset has been moved to:", cifar100_dataset_folder_path)

# Check if the dataset is nested in a subdirectory (e.g., 'cifar100-preprocessed')
nested_folder = os.path.join(cifar100_dataset_folder_path, 'cifar100-preprocessed')
if os.path.exists(nested_folder):
    print("Found nested dataset folder. Moving contents to the root directory...")
    for item in os.listdir(nested_folder):
        source_item_path = os.path.join(nested_folder, item)
        destination_item_path = os.path.join(cifar100_dataset_folder_path, item)

        # Move the item
        shutil.move(source_item_path, destination_item_path)

    # Remove the now-empty nested folder
    os.rmdir(nested_folder)
    print("Contents moved to the root directory.")

# Verify the contents of the target directory
print("Contents of the target directory:", os.listdir(cifar100_dataset_folder_path))

100%|██████████| 2.62G/2.62G [02:17<00:00, 20.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ibraheemmoosa/cifar100-256x256/versions/1
Dataset has been moved to: /content/drive/MyDrive/Datasets/cifar100
Found nested dataset folder. Moving contents to the root directory...
Contents moved to the root directory.
Contents of the target directory: ['test', 'train']


In [6]:
import os
import re
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [1]:
!nvidia-smi

Sat Jan 11 11:10:28 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
print(f'files in the dataset folder path{os.listdir(cifar100_dataset_folder_path)}')

files in the dataset folder path['test', 'train']


In [9]:

# Define the dataset path
cifar100_dataset_folder_path = '/content/drive/MyDrive/Datasets/cifar100'

# List files in the train folder
train_folder = os.path.join(cifar100_dataset_folder_path, 'train')
train_files = os.listdir(train_folder)
print("Files in 'train' folder:", len(train_files))

# List files in the test folder
test_folder = os.path.join(cifar100_dataset_folder_path, 'test')
test_files = os.listdir(test_folder)
print("Files in 'test' folder:", len(test_files))

Files in 'train' folder: 50000
Files in 'test' folder: 10000


In [41]:
import os
import re
import shutil

def organize_dataset_into_class_directories(root_dir):
    """
    Organizes a dataset into class subdirectories based on labels extracted from filenames.

    Args:
        root_dir (str): Path to the dataset folder (e.g., '/content/drive/MyDrive/Datasets/cifar100/train').
    """
    # Get all image file paths in the root directory
    image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.png')]

    # Ensure images are found
    if not image_paths:
        raise RuntimeError(f"No images found in {root_dir}. Make sure the folder contains .png files.")

    # Create class directories
    for class_label in range(100):  # CIFAR-100 has 100 classes
        class_dir = os.path.join(root_dir, f"class{class_label}")
        os.makedirs(class_dir, exist_ok=True)

    # Move images to their respective class folders
    for image_path in image_paths:
        # Extract the filename from the full path
        filename = os.path.basename(image_path)

        # Use a regular expression to find the 2-digit label in the filename
        match = re.search(r'\d{2}', filename)
        if not match:
            raise ValueError(f"Filename {filename} does not contain a valid 2-digit label.")

        # Extract the label and convert it to an integer
        label = int(match.group())

        # Define the destination directory for this image
        class_dir = os.path.join(root_dir, f"class{label}")

        # Move the image to the class directory
        shutil.move(image_path, os.path.join(class_dir, filename))

    print(f"Dataset organized into class directories in {root_dir}.")

# Define paths to your train and test folders
train_folder = '/content/drive/MyDrive/Datasets/cifar100/train'
test_folder = '/content/drive/MyDrive/Datasets/cifar100/test'

# Organize the train and test datasets
organize_dataset_into_class_directories(train_folder)
organize_dataset_into_class_directories(test_folder)

Dataset organized into class directories in /content/drive/MyDrive/Datasets/cifar100/train.
Dataset organized into class directories in /content/drive/MyDrive/Datasets/cifar100/test.


#YOLOv8 medium model

In [39]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 42.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 classification model (medium size)
model = YOLO('yolov8m-cls.pt')  # 'm' stands for medium size

# Train the model on your dataset
results = model.train(
    data='/content/drive/MyDrive/Datasets/cifar100',  # Path to your dataset
    epochs=10,                # Number of epochs
    imgsz=224,                # Image size
    batch=64,                 # Batch size
    name='yolov8m_cls_cifar100',  # Name of the training run
    project='classification', # Project name
    optimizer='Adam',         # Optimizer
    lr0=0.001,               # Initial learning rate
    patience=3,              # Early stopping patience
    device='0'               # Use GPU (set to 'cpu' if no GPU is available)
)



Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/content/drive/MyDrive/Datasets/cifar100, epochs=10, time=None, patience=3, batch=64, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=8, project=classification, name=yolov8m_cls_cifar1002, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

/usr/local/lib/python3.10/dist-packages/ultralytics/utils/torch_utils.py:262: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/torch_utils.py:267: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operati

AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/Datasets/cifar100/train... 50000 images, 0 corrupt: 100%|██████████| 50000/50000 [03:05<00:00, 269.80it/s]


train: New cache created: /content/drive/MyDrive/Datasets/cifar100/train.cache


...


      Epoch    GPU_mem       loss  Instances       Size


  0%|          | 0/782 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [01:52<00:00,  1.43s/it]

                   all          0     0.0994



      Epoch    GPU_mem       loss  Instances       Size


  0%|          | 0/782 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
       7/10      2.59G      3.919         64        224:  35%|███▌      | 274/782 [04:24<05:37,  1.51it/s]

In [ ]:
# Evaluate the model on the validation set
metrics = model.val()

# Export the model to ONNX format (optional)
model.export(format='onnx')